<a href="https://colab.research.google.com/github/VicthorAstudillo/03MAIR-Algoritmos-de-Optimizacion--2025/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos(V2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Victor Daniel Astudillo Vallejo  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---/tree/master/TrabajoPractico<br>
Google Colab: https://colab.research.google.com/drive/1_Cu3TB7R5xHCT8LpdxtoQksTdFbhem2i?usp=sharing <br>
Problema:
>1. Sesiones de doblaje <br>


Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible.

Número de actores: 10

Número de tomas : 30

Actores/Tomas : https://bit.ly/36D8IuK
- 1 indica que el actor participa en la toma
- 0 en caso contrario







                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

In [2]:
#Respuesta


La solución puede representarse como una asignación de tomas por día, asegurando que no se graben más de 6 tomas por día, en caso de que uno de los actores aparesca en todas las escenas
Por ejemplo:

Día 1: Tomas [1,2,3,4,5]

Día 2: Tomas [6,7,8,9]

...

La **funcion objetivo**  cuenta el numero de actores que se necesaitan por dia y los suma al final, asi tratamos de minimizar dicho número.

Las **restricciones** deben ser las siguientes:
1. No se pueden asignar más de 6 tomas a un solo día.


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

In [ ]:
#Respuesta


El problema pertenece a la clase NP-difícil, ya que es similar a problemas de partición y agrupamiento óptimos. Al existir 30 tomas y estas pueden combinarse en diferentes días la cantidad de asignaciones posibles es $O(6^{30})$


#Diseño
- ¿Que técnica utilizo? ¿Por qué?

In [ ]:
#Respuesta

Se ha utilizado la metaheurística de Recocido Simulado (Simulated Annealing) ya que el espacio de soluciones es muy grande, es ideal para este tipo de problemas porque puede escapar de óptimos locales y explorar mejor el espacio de soluciones. Además se combinó con la simulación múltiple para mejorar resultados.

#Resolución

In [256]:
import numpy as np
import random
import math

# ============================================================================
# 1. MATRIZ DE PARTICIPACIÓN ACTOR - TOMA
# ============================================================================

matriz = np.array([
    [1,1,1,1,1,0,0,0,0,0], [0,0,1,1,1,0,0,0,0,0], [0,1,0,0,1,0,1,0,0,0],
    [1,1,0,0,0,0,1,1,0,0], [0,1,0,1,0,0,0,1,0,0], [1,1,0,1,1,0,0,0,0,0],
    [1,1,0,0,1,1,0,0,0,0], [1,1,0,0,0,1,0,0,0,0], [1,1,0,1,0,0,0,0,0,0],
    [1,1,0,0,0,1,0,0,1,0], [1,1,1,0,1,0,0,1,0,0], [1,1,1,1,0,1,0,0,0,0],
    [1,0,0,1,1,0,0,0,0,0], [1,0,1,0,0,1,0,0,0,0], [1,1,0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0,0,1], [1,0,1,0,0,0,0,0,0,0], [0,0,1,0,0,1,0,0,0,0],
    [1,0,1,0,0,0,0,0,0,0], [1,0,1,1,1,0,0,0,0,0], [0,0,0,0,0,1,0,1,0,0],
    [1,1,1,1,0,0,0,0,0,0], [1,0,1,0,0,0,0,0,0,0], [0,0,1,0,0,1,0,0,0,0],
    [1,1,0,1,0,0,0,0,0,1], [1,0,1,0,1,0,0,0,1,0], [0,0,0,1,1,0,0,0,0,0],
    [1,0,0,1,0,0,0,0,0,0], [1,0,0,0,1,1,0,0,0,0], [1,0,0,1,0,0,0,0,0,0]
])

# ============================================================================
# 2. FUNCIÓN OBJETIVO
# ============================================================================
def objective(schedule, matriz):
    """
    Para cada día:
      - Se seleccionan las filas (tomas) correspondientes (recordando que las tomas se
        numeran de 1 a 30, por lo que se ajusta a 0-index)
      - Se utiliza np.logical_or.reduce para obtener la unión (OR) de las filas, lo que
        indica, para cada actor, si participó al menos en una toma del día.
      - Se cuenta el número de actores (1's en la unión)

    Finalmente se suma el resultado para todos los días

    """
    total_cost = 0
    for day in schedule:
        # Convertir los números de toma (1-indexados) a índices (0-indexados)
        indices = np.array(day) - 1
        # Extraer las filas correspondientes a las tomas del día
        day_matrix = matriz[indices-1, :]
        # Calcular la unión lógica de las filas para obtener, por cada actor,
        # si participó al menos en una toma del día.
        union = np.logical_or.reduce(day_matrix, axis=0)
        cost_day = int(np.sum(union))  # Suma de 1's (actores que participaron)
        total_cost += cost_day
    return total_cost

# ============================================================================
# 3. GENERAR SOLUCIÓN INICIAL
# ============================================================================
def generate_initial_schedule():
    """
    Genera una solución inicial aleatoria particionando las 30 tomas en 5 días (6 tomas cada uno)
    """
    shots = np.arange(1, 31)  # Tomás numeradas del 1 al 30
    np.random.shuffle(shots)
    schedule = [shots[i*6:(i+1)*6].tolist() for i in range(5)]
    return schedule

# ============================================================================
# 4. OPERADOR DE VECINDAD
# ============================================================================
def neighbor(schedule):
    """
    Genera una solución vecina intercambiando dos tomas de días distintos
    """
    new_schedule = [day[:] for day in schedule]  # Copia de la solución actual
    day1, day2 = random.sample(range(5), 2)       # Selecciona dos días aleatoriamente
    idx1 = random.randrange(len(new_schedule[day1]))
    idx2 = random.randrange(len(new_schedule[day2]))
    new_schedule[day1][idx1], new_schedule[day2][idx2] = new_schedule[day2][idx2], new_schedule[day1][idx1]
    return new_schedule

# ============================================================================
# 5. ALGORTIMO DE RECOCIDO SIMULADO
# ============================================================================
def simulated_annealing(matriz, initial_temp=10000, alpha=0.99, temp_min=1e-3, max_iter=1000000):
    """
    Aplica recocido simulado para minimizar la función objetivo
    """
    current_schedule = generate_initial_schedule_np()
    current_obj = objective(current_schedule, matriz)
    best_schedule = current_schedule
    best_obj = current_obj
    temp = initial_temp
    iter_count = 0

    while temp > temp_min and iter_count < max_iter:
        new_schedule = neighbor(current_schedule)
        new_obj = objective(new_schedule, matriz)
        delta = new_obj - current_obj

        # Aceptamos la nueva solución si mejora o, si empeora, con cierta probabilidad
        if delta < 0 or random.random() < math.exp(-delta / temp):
            current_schedule = new_schedule
            current_obj = new_obj
            if current_obj < best_obj:
                best_schedule = current_schedule
                best_obj = current_obj

        temp *= alpha
        iter_count += 1

    return best_schedule, best_obj

# ============================================================================
# 6. EJECUCIÓN MULTIPLE PARA GUARDAR LA MEJOR SOLUCIÓN
# ============================================================================
def run_multiple_attempts(num_attempts):
    best_overall_schedule = None
    best_overall_obj = float('inf')

    for i in range(num_attempts):
        schedule, cost = simulated_annealing(matriz)
        #print(f"Intento {i+1}: costo = {cost}")
        if cost < best_overall_obj:
            best_overall_obj = cost
            best_overall_schedule = schedule

    return best_overall_schedule, best_overall_obj

if __name__ == "__main__":
    # Ejecuta el recocido simulado múltiples veces y guarda la mejor solución
    num_attempts = 1000
    best_schedule, best_obj = run_multiple_attempts(num_attempts)

    print("\nMejor solución encontrada:")
    print("Costo:", best_obj)
    for i, day in enumerate(best_schedule):
        print(f"Día {i+1}: {day}")



Mejor solución encontrada:
Costo: 27
Día 1: [2, 21, 23, 12, 6, 3]
Día 2: [16, 22, 30, 5, 8, 4]
Día 3: [10, 7, 14, 1, 29, 28]
Día 4: [24, 18, 20, 15, 19, 25]
Día 5: [26, 13, 27, 17, 11, 9]
